The first step is to import the Notebook loader code, which enables the import of notebooks as if they were python code.

In [ ]:
import daipy.nb_loader

Next, let's import numpy and matplotlib, because we will most certainly use them.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

Let's import the Camera class from its notebook, which simplifies getting images from the USB Camera

In [ ]:
from daipy.cam import Camera
cam = Camera()

We should make sure that it's working... Let's capture a frame and display it!

In [ ]:
rgb = cam.get_rgb()
print("Camera image shape: " + str(rgb.shape))
plt.imshow(rgb)
plt.show()

Let's import the DPU module and use it to load the bitstream and the DNN model!

In [ ]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")
overlay.load_model("yolo/dk_yolov3_voc_416_416.xmodel")

We should take a look at the input and output tensors and make sure it lines up with what we expect.

In [ ]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
print("Input Tensors: " + str(inputTensors))
outputTensors = dpu.get_output_tensors()
print("Output Tensors: " + str(outputTensors))

Let's also take a look at their shapes

In [ ]:
inShapes = []
for tensor in inputTensors:
    inShapes.append(tuple(tensor.dims))
print("Input Shapes: " + str(inShapes))
outShapes = []
for tensor in outputTensors:
    outShapes.append(tuple(tensor.dims))
print("Output Shapes: " + str(outShapes))

im_res = (inShapes[0][1], inShapes[0][2])
print("Required input image resolution: " + str(im_res))

Create the input/output buffers

In [ ]:
output_data = []
for shape in outShapes:
    output_data.append(np.empty(shape, dtype=np.float32, order="C"))
input_data = []
for shape in inShapes:
    input_data.append(np.empty(shape, dtype=np.float32, order="C"))
image = input_data[0]

Create a function for easy inferencing

In [ ]:
from yolo.evaluator import Evaluator
evaluator = Evaluator()

In [ ]:
def detect(img, label="person", display=False):
    preprocessed = np.array(evaluator.pre_process(img, im_res), dtype=np.float32)
    image[0,...] = preprocessed.reshape(image.shape[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    image_size = img.shape[:2]
    boxes, scores, classes = evaluator.evaluate(output_data, image_size)
    if display:
        evaluator.draw_boxes(img, boxes, scores, classes)
        px = 1/plt.rcParams['figure.dpi']  # pixel in inches
        _, ax = plt.subplots(1, figsize=(image_size[0]*px*2,image_size[1]*px*2))
        _ = ax.imshow(img)
    for i in range(len(classes)):
        if evaluator.class_names[classes[i]] == label:
            return boxes[i]
    return None

Create a function for publishing the resutls

In [ ]:
from daipy.box_publisher import BoxPublisher
pub = BoxPublisher()

In [ ]:
def publish_box(box, debug = False):
    if box is not None:
        # box is top, left, bottom, right
        h, w, _ = img.shape
        [top, left, bottom, right] = box
        nbox = [
            top/h,
            left/w,
            bottom/h,
            right/w
        ]
        pub.publish(nbox, debug)

Now we can run the inferencing until the client cancels

In [ ]:
while pub.active:
    img = cam.get_rgb()
    box = detect(img, display=True)
    publish_box(box, debug=True)

Finally, we should clean up

In [ ]:
cam.close()
pub.close()